In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import glob
from lib.llama3.reference_impl.generation import Llama
import subprocess
from torchtune.models.llama3_1 import llama3_1_8b
from torchtune.training.checkpointing import FullModelHFCheckpointer

model_dir = subprocess.run(
    "HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download NousResearch/Hermes-2-Theta-Llama-3-8B",
    shell=True,
    capture_output=True,
    text=True,
).stdout.strip()

checkpointer = FullModelHFCheckpointer(
    checkpoint_dir=model_dir,
    checkpoint_files=glob.glob(f"{model_dir}/*.safetensors"),
    output_dir=model_dir,
    model_type='LLAMA3' # type: ignore
)
state_dict = checkpointer.load_checkpoint()

model = llama3_1_8b()
model.load_state_dict(state_dict)

RuntimeError: Error(s) in loading state_dict for TransformerDecoder:
	Missing key(s) in state_dict: "tok_embeddings.weight", "layers.0.attn.q_proj.weight", "layers.0.attn.k_proj.weight", "layers.0.attn.v_proj.weight", "layers.0.attn.output_proj.weight", "layers.0.mlp.w1.weight", "layers.0.mlp.w2.weight", "layers.0.mlp.w3.weight", "layers.0.sa_norm.scale", "layers.0.mlp_norm.scale", "layers.1.attn.q_proj.weight", "layers.1.attn.k_proj.weight", "layers.1.attn.v_proj.weight", "layers.1.attn.output_proj.weight", "layers.1.mlp.w1.weight", "layers.1.mlp.w2.weight", "layers.1.mlp.w3.weight", "layers.1.sa_norm.scale", "layers.1.mlp_norm.scale", "layers.2.attn.q_proj.weight", "layers.2.attn.k_proj.weight", "layers.2.attn.v_proj.weight", "layers.2.attn.output_proj.weight", "layers.2.mlp.w1.weight", "layers.2.mlp.w2.weight", "layers.2.mlp.w3.weight", "layers.2.sa_norm.scale", "layers.2.mlp_norm.scale", "layers.3.attn.q_proj.weight", "layers.3.attn.k_proj.weight", "layers.3.attn.v_proj.weight", "l